##Imports

In [1]:
import re
import pandas as pd
import json
import numpy as np
import seaborn as sns
import nltk
from bertopic import BERTopic

/home/lucas/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Extração de Dados

In [2]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename).dropna().reset_index()[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

# Pré-Processamento

In [3]:
tweets = df_tweets_candidatos('./datasets/dataset.json')


# Adicionar nomes dos candidatos no dataframe
for index, row in tweets.iterrows():
    candidato = ''
    if row.author_id == 2670726740:
      candidato = 'lula'
    elif row.author_id == 128372940:
      candidato = 'bolsonaro'
    elif row.author_id == 33374761:
      candidato = 'ciro'
    else:
      candidato = 'n/d'
    tweets.at[index, 'candidato'] = candidato

tweets = tweets.sample(frac=1)
print(f'Número de tweets: {len(tweets)}\n')
print(tweets.head())

Número de tweets: 300

      author_id      conversation_id                created_at  \
74   2670726740  1558834809882943488 2022-08-14 15:15:55+00:00   
286   128372940  1555964162181943296 2022-08-06 17:08:59+00:00   
252   128372940  1557892369990795264 2022-08-12 00:51:00+00:00   
11   2670726740  1560032266042019840 2022-08-17 22:34:10+00:00   
165    33374761  1559377178495787008 2022-08-16 03:11:05+00:00   

                                                  text                   id  \
74   Beagá, semana que vem tem Lula na cidade! Já r...  1558834809882943488   
286  - Hoje, em Recife / Pernambuco (06/08/2022). 🇧...  1555964162181943296   
252  - O Brasil já tem sua carta pela democracia: a...  1557892373379702784   
11   Atenção BH: amanhã tem ato na cidade! O evento...  1560032266042019840   
165  Estamos propondo um novo sistema previdenciári...  1559377178495787008   

     candidato  
74        lula  
286  bolsonaro  
252  bolsonaro  
11        lula  
165       ciro  


In [4]:
nltk.download('stopwords')
pt_stop = set(nltk.corpus.stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import re
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

def preprocess_text(document):

        # removing urls
        document = re.sub(r'http\S+', '', document)

        # removing mentions
        document = re.sub(r'\B@\w+', '', document)

        # remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # converting to lowercase
        document = document.lower()

        # lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in pt_stop]
        tokens = [word for word in tokens if len(word) > 3]

        return ' '.join(tokens)

# Modelagem de Tópicos

In [6]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /home/lucas/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lucas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# create model 
model = BERTopic(language="multilingual", verbose=True, min_topic_size=3, top_n_words=10, calculate_probabilities=True)

# convert to list 
docs = tweets.text.to_list()

# Pre-processamento
for i, doc in enumerate(docs):
    docs[i] = preprocess_text(doc)

tweets_topic, probabilities = model.fit_transform(docs)

Batches: 100%|██████████| 10/10 [00:13<00:00,  1.36s/it]
2022-08-25 21:43:42,103 - BERTopic - Transformed documents to Embeddings
2022-08-25 21:43:53,610 - BERTopic - Reduced dimensionality
2022-08-25 21:43:53,942 - BERTopic - Clustered reduced embeddings


Adicionando o tópico de cada tweet no Dataframe

In [8]:
topic_names = model.generate_topic_labels(nr_words=4)

# Array indicando o tópico de cada tweet (numérico -> índice do array "topic_names" + 1)
# print(tweets_topic)
# Array com todos os tópicos nomeados
# print(topic_names)

for index, tweet_topic in enumerate(tweets_topic):
    tweets.at[index + 1, 'topico'] = topic_names[tweet_topic + 1]

tweets.head()

,author_id,conversation_id,created_at,text,id,candidato,topico
74,2.670727e+09,1.558835e+18,2022-08-14 15:15:55+00:00,"Beagá, semana que vem tem Lula na cidade! Já r...",1.558835e+18,lula,1_melhor_brasildaesperança_esperança_brasil
286,1.283729e+08,1.555964e+18,2022-08-06 17:08:59+00:00,"- Hoje, em Recife / Pernambuco (06/08/2022). 🇧...",1.555964e+18,bolsonaro,13_central_pergunta_cironorodaviva_banco
252,1.283729e+08,1.557892e+18,2022-08-12 00:51:00+00:00,- O Brasil já tem sua carta pela democracia: a...,1.557892e+18,bolsonaro,21_auxílio_emergencial_dezembro_agora
11,2.670727e+09,1.560032e+18,2022-08-17 22:34:10+00:00,Atenção BH: amanhã tem ato na cidade! O evento...,1.560032e+18,lula,7_eleição_voto_falar_café
165,3.337476e+07,1.559377e+18,2022-08-16 03:11:05+00:00,Estamos propondo um novo sistema previdenciári...,1.559377e+18,ciro,1_melhor_brasildaesperança_esperança_brasil


Abaixo, os tópicos identificados entre todos os tweets

In [9]:
freq = model.get_topic_info()
print(freq)

    Topic  Count                                               Name
0      -1     46                      -1_ciro_brasil_lula_reeleição
1       0     25                       0_luladay_noite_gettr_baixar
2       1     22        1_melhor_brasildaesperança_esperança_brasil
3       2     17                2_milhões_famílias_safra_desemprego
4       3     16                        3_deus_cristãos_crime_irmão
5       4     14                    4_pequenos_custou_quanto_básico
6       5     14               5_redução_energia_combustíveis_preço
7       6     14                        6_link_lula_equipelula_vivo
8       7     11                          7_eleição_voto_falar_café
9       8      9            8_saudades_democracia_carta_democrático
10      9      8                 9_programa_renda_mínima_distribuir
11     10      7           10_quer_merecemos_responsabilidade_guiar
12     11      7                          11_todos_vivo_vice_beonce
13     12      7              12_estímulo_compet

# Análise de Sentimentos

# Similaridade de Textos

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
def monta_docs_tweets(df, topico, coluna_alvo, candidatos):
    lista_tweets = []
    #df_topico = df[df['topico'] == topico]
    df_topico = df

    for cand in candidatos:
        df_cand = df_topico[df_topico['candidato'] == cand]
        lista_tweets_cand = df_cand[coluna_alvo].to_list()
        lista_tweets.append(lista_tweets_cand)
    
    docs = np.concatenate(lista_tweets)
    return docs

def monta_docs_concat_topico(df, topico, coluna_alvo, candidatos):
    docs = []
    df_topico = df[(df['topico'] == topico)]
    for cand in candidatos:
        df_topico_cand = df_topico[df_topico['candidato'] == cand]
        texto = ' '.join(df_topico_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs


def monta_docs_concat_candidato(df, coluna_alvo, candidatos):
    docs = []
    for cand in candidatos:
        df_cand = df[df['candidato'] == cand]
        texto = ' '.join(df_cand[coluna_alvo].to_list())
        docs.append(texto)
    return docs

stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')

def normaliza_tweet(tweet):
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet, re.I|re.A)
    tweet = tweet.lower()
    tweet = tweet.strip()
    tokens = nltk.word_tokenize(tweet)
    tokens_filtrados = [token for token in tokens if token not in stop_words_ptbr]
    tweet_tokens = ' '.join(tokens_filtrados)
    return tweet_tokens

def vetoriza_docs(lista_docs):
    corpus_vetorizado = np.vectorize(lista_docs)
    return corpus_vetorizado

def gera_matriz_tfidf(corpus_vetorizado):
    tf = TfidfVectorizer(ngram_range=(1, 2), min_df=1)
    matriz_tfidf = tf.fit_transform(corpus_vetorizado)
    return matriz_tfidf

def similaridade_pares(matriz_tfidf):
    dict_docs_sim = cosine_similarity(matriz_tfidf)
    df_sim = pd.DataFrame(dict_docs_sim)
    return df_sim

def renomear_colunas(df, candidatos):
    df.columns = candidatos
    return df

def calcula_similaridade_topico(df, topico, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_topico(df, topico, coluna_alvo, candidatos)
    normaliza_tweets = np.vectorize(normaliza_tweet)
    corpus_normalizado = normaliza_tweets(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)
    df_sim_cands = renomear_colunas(df_sim, candidatos)

    return df_sim_cands

def calcula_similaridade_candidatos(df, coluna_alvo, candidatos):
    lista_docs = monta_docs_concat_candidato(df, coluna_alvo, candidatos)
    normaliza_tweets = np.vectorize(normaliza_tweet)
    corpus_normalizado = normaliza_tweets(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_normalizado)
    df_sim = similaridade_pares(matriz_tfidf)
    df_sim_cands = renomear_colunas(df_sim, candidatos)

    return df_sim

def calcula_similaridade_topicos(df, topicos, coluna_alvo, candidatos):
    dfs_sim = []
    for topico in topicos:
        dfs_sim.append(calcula_similaridade_topico(df, topico, coluna_alvo, candidatos))
        
    sim_dict = dict(zip(topicos, dfs_sim))
    return sim_dict

In [12]:
df_freq = model.get_topic_info()
topicos_mais_frequentes = df_freq['Name'].to_list()[0:4]
topicos_mais_frequentes

['-1_ciro_brasil_lula_reeleição',
 '0_luladay_noite_gettr_baixar',
 '1_melhor_brasildaesperança_esperança_brasil',
 '2_milhões_famílias_safra_desemprego']

In [13]:
candidatos = ["lula", "bolsonaro", "ciro"]
analise_similaridade = calcula_similaridade_topicos(tweets, topicos_mais_frequentes, "text", candidatos)
for topico in analise_similaridade.keys():
    print(f"Para tópico = {topico}")
    print(analise_similaridade[topico])
    print('----------------------------------')

Para tópico = -1_ciro_brasil_lula_reeleição
       lula  bolsonaro      ciro
0  1.000000   0.012569  0.070754
1  0.012569   1.000000  0.028200
2  0.070754   0.028200  1.000000
----------------------------------
Para tópico = 0_luladay_noite_gettr_baixar
       lula  bolsonaro      ciro
0  1.000000   0.025657  0.073263
1  0.025657   1.000000  0.014557
2  0.073263   0.014557  1.000000
----------------------------------
Para tópico = 1_melhor_brasildaesperança_esperança_brasil
       lula  bolsonaro      ciro
0  1.000000   0.007351  0.041668
1  0.007351   1.000000  0.020782
2  0.041668   0.020782  1.000000
----------------------------------
Para tópico = 2_milhões_famílias_safra_desemprego
       lula  bolsonaro      ciro
0  1.000000   0.007339  0.024608
1  0.007339   1.000000  0.011247
2  0.024608   0.011247  1.000000
----------------------------------


In [14]:
analise_sim_cand = calcula_similaridade_candidatos(tweets, "text", candidatos)
analise_sim_cand

,lula,bolsonaro,ciro
0,1.000000,0.333256,0.365497
1,0.333256,1.000000,0.293330
2,0.365497,0.293330,1.000000
